In [3]:
import cv2
import numpy as np

# Load your saved images
image_1 = cv2.imread('C:\\Users\\Darshan\\Downloads\\Computer Vision\\box.png', cv2.IMREAD_GRAYSCALE)
image_2 = cv2.imread('C:\\Users\\Darshan\\Downloads\\Computer Vision\\box_in_scene.png', cv2.IMREAD_GRAYSCALE)

# Resize second image to match the size of first
image_2 = cv2.resize(image_2, (image_1.shape[1], image_1.shape[0]))

# SIFT detector
sift = cv2.SIFT_create()
keypoints_1, descriptors_1 = sift.detectAndCompute(image_1, None)
keypoints_2, descriptors_2 = sift.detectAndCompute(image_2, None)

# FLANN matcher
index_params = dict(algorithm=1, trees=5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(descriptors_1, descriptors_2, k=2)

# Lowe's ratio test
good_matches = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good_matches.append(m)

# Homography and Warping
if len(good_matches) > 10:
    src_pts = np.float32([keypoints_1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints_2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    
    h, w = image_1.shape
    aligned_image = cv2.warpPerspective(image_2, M, (w, h))

    # Display images using OpenCV
    cv2.imshow('Image 1 (Reference)', image_1)
    cv2.imshow('Image 2 (Scene)', image_2)
    cv2.imshow('Aligned Image (Stitched)', aligned_image)

    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Not enough matches found to compute homography.")